In [4]:
import preprocess as pp
import configs
import persister
import relation
import lda

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize
from nltk.tree import Tree

inited stanford CoreNLP client, dont forget to close it!
closed stanford CoreNLP client!


In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [10]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Guangdong University of Foreign Studies is located in Guangzhou.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Tokenize: ['Guangdong', 'University', 'of', 'Foreign', 'Studies', 'is', 'located', 'in', 'Guangzhou', '.']
Part of Speech: [('Guangdong', 'NNP'), ('University', 'NNP'), ('of', 'IN'), ('Foreign', 'NNP'), ('Studies', 'NNPS'), ('is', 'VBZ'), ('located', 'JJ'), ('in', 'IN'), ('Guangzhou', 'NNP'), ('.', '.')]
Named Entities: [('Guangdong', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('of', 'ORGANIZATION'), ('Foreign', 'ORGANIZATION'), ('Studies', 'ORGANIZATION'), ('is', 'O'), ('located', 'O'), ('in', 'O'), ('Guangzhou', 'CITY'), ('.', 'O')]
Constituency Parsing: (ROOT
  (S
    (NP
      (NP (NNP Guangdong) (NNP University))
      (PP (IN of)
        (NP (NNP Foreign) (NNPS Studies))))
    (VP (VBZ is)
      (ADJP (JJ located)
        (PP (IN in)
          (NP (NNP Guangzhou)))))
    (. .)))
Dependency Parsing: [('ROOT', 0, 7), ('compound', 2, 1), ('nsubjpass', 7, 2), ('case', 5, 3), ('compound', 5, 4), ('nmod', 2, 5), ('auxpass', 7, 6), ('case', 9, 8), ('nmod', 7, 9), ('punct', 7, 10)]

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [ ]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [ ]:
texts = [' '.join(pp.preprocess_abstract(a)) for a in tmp]
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [7]:
texts = persister.load_json(configs.NEWSDATA)
newssent = persister.load_json(configs.NEWSSENT)
newssenttoken = persister.load_json(configs.NEWSSENTTOKEN)

In [ ]:
# load lda res
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)

In [ ]:
relation.CLI.pos_tag(newssent[432][13])

In [ ]:
dparse = relation.CLI.dependency_parse(newssent[432][13])

In [ ]:
from nltk.tokenize.stanford import StanfordTokenizer
er=StanfordTokenizer()

In [ ]:
cparse = relation.CLI.parse(newssent[432][13])
print(cparse)
t = Tree.fromstring(cparse)
t.draw()

In [ ]:
t.triples()

In [ ]:
s = relation.get_sent_idx(newssenttoken[432], "graphic")

In [ ]:
s

In [10]:
newssent[0]

['In regards to fractal commpression, I have seen 2 fractal compressed "movies".',
 'They were both fairly impressive.',
 'The first one was a 64 gray scale "movie" of Casablanca, it was 1.3MB and had 11 minutes of 13 fps video.',
 'It was a little grainy but not bad at all.',
 'The second one I saw was only 3 minutes but it had 8 bit color with 10fps and measured in at 1.2MB.',
 'I consider the fractal movies a practical thing to explore.',
 'But unlike many  other formats out there, you do end up losing resolution.',
 'I don\'t know what kind of software/hardware was used for creating the "movies" I saw but the guy that showed them to me said it took 5-15 minutes per frame to generate.',
 'But as I said above playback was 10 or more frames per second.',
 'And how else could you put 11 minutes on one floppy disk?']

In [2]:
import json
with open("exampleres.json",encoding="utf8")as f:
    res = json.load(f)
print(res)

[{'sentences': [{'index': 0, 'parse': "(ROOT\n  (S\n    (SBAR (IN In)\n      (S\n        (VP (VBZ regards)\n          (PP (TO to)\n            (NP (JJ fractal) (NN commpression))))))\n    (, ,)\n    (NP (PRP I))\n    (VP (VBP have)\n      (VP (VBN seen)\n        (S\n          (NP (CD 2))\n          (ADJP (JJ fractal))\n          (S\n            (VP (VBN compressed)\n              (S (`` ``)\n                (NP (NNS movies))\n                ('' '')))))))\n    (. .)))", 'basicDependencies': [{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 9, 'dependentGloss': 'seen'}, {'dep': 'case', 'governor': 2, 'governorGloss': 'regards', 'dependent': 1, 'dependentGloss': 'In'}, {'dep': 'nmod', 'governor': 9, 'governorGloss': 'seen', 'dependent': 2, 'dependentGloss': 'regards'}, {'dep': 'case', 'governor': 5, 'governorGloss': 'commpression', 'dependent': 3, 'dependentGloss': 'to'}, {'dep': 'amod', 'governor': 5, 'governorGloss': 'commpression', 'dependent': 4, 'dependentGloss':

In [5]:
t = Tree.fromstring(res[0]["sentences"][0]["parse"])

In [12]:
res[0]["sentences"][0]["tokens"]

[{'index': 1,
  'word': 'In',
  'originalText': 'In',
  'characterOffsetBegin': 0,
  'characterOffsetEnd': 2,
  'pos': 'IN',
  'before': '',
  'after': ' '},
 {'index': 2,
  'word': 'regards',
  'originalText': 'regards',
  'characterOffsetBegin': 3,
  'characterOffsetEnd': 10,
  'pos': 'VBZ',
  'before': ' ',
  'after': ' '},
 {'index': 3,
  'word': 'to',
  'originalText': 'to',
  'characterOffsetBegin': 11,
  'characterOffsetEnd': 13,
  'pos': 'TO',
  'before': ' ',
  'after': ' '},
 {'index': 4,
  'word': 'fractal',
  'originalText': 'fractal',
  'characterOffsetBegin': 14,
  'characterOffsetEnd': 21,
  'pos': 'JJ',
  'before': ' ',
  'after': ' '},
 {'index': 5,
  'word': 'commpression',
  'originalText': 'commpression',
  'characterOffsetBegin': 22,
  'characterOffsetEnd': 34,
  'pos': 'NN',
  'before': ' ',
  'after': ''},
 {'index': 6,
  'word': ',',
  'originalText': ',',
  'characterOffsetBegin': 34,
  'characterOffsetEnd': 35,
  'pos': ',',
  'before': '',
  'after': ' '},
 {

In [9]:
t.pretty_print()

                                                      ROOT                                                   
                                                       |                                                      
                                                       S                                                     
              _________________________________________|___________________________________________________   
             |                                 |   |        VP                                             | 
             |                                 |   |    ____|____                                          |  
             |                                 |   |   |         VP                                        | 
             |                                 |   |   |     ____|______________                           |  
            SBAR                               |   |   |    |                   S                          | 
  ____